# LeetCode BFS Serialization Format

https://support.leetcode.com/hc/en-us/articles/360011883654-What-does-1-null-2-3-mean-in-binary-tree-representation-

BFS order, but null nodes' children get skipped on the next level so that you don't have a bunch of extra nulls.

BFS serialization is less optimal than pre-order DFS, but more human readable, which is why LeetCode uses it for problems.

# Basic Setup

This code defines the `Node` class we'll use throughout and also gives a simple BFS serialization/deserialization to support a LeetCode like format.  I used # for null with commas in between.

To keep it simple/optimal:
 - I let the trailing comma stay in
 - I didn't remove the extra `#` you get at the end because the algorithm doesn't know the last level is fake
   - not trivial to remove those

In [1]:
import java.util.Queue;
import java.util.LinkedList;
import java.util.Stack;
import java.text.StringCharacterIterator;

class Node {
    int value;
    Node left;
    Node right;
    
    public Node(int value) {
        this.value = value;
    }
    
    public Node(int value, Node parent) {
        this.value = value;
        if (parent.left == null) {
            parent.left = this;
        }
        else {
            parent.right = this;
        }
    }
    
    @Override
    public String toString() {
        StringBuilder output = new StringBuilder();
        Queue<Node> q = new LinkedList<>();
        q.add(this);
        output.append(value);
        output.append(',');
        
        while (!q.isEmpty()) {
            Node next = q.poll();
            
            if (next.left == null) {
                output.append('#');
            }
            else {
                output.append(next.left.value);
                q.add(next.left);
            }
            output.append(',');
            
            if (next.right == null) {
                output.append('#');
            }
            else {
                output.append(next.right.value);
                q.add(next.right);
            }
            output.append(',');
        }
        
        return output.toString();
    }
    
    private static Node nextValue(StringCharacterIterator iter) {
        if (iter.current() == '#') {
            iter.next();
            iter.next();
            return null;
        }
        if (iter.current() == StringCharacterIterator.DONE) {
            return null;
        }
        
        int currentValue = 0;
        int sign = iter.current() == '-' ? -1 : 1;
        if (sign == -1) iter.next();
        while (iter.current() != ',' && iter.current() != StringCharacterIterator.DONE) {
            currentValue = currentValue * 10 + iter.current() - '0';
            iter.next();
        }
        iter.next();
        
        return new Node(currentValue * sign);
    }
    
    public static Node parseNode(String text) {
        if (text == null || text.length() == 0) {
            return null;
        }
        
        Queue<Node> q = new LinkedList<>();
        StringCharacterIterator iter = new StringCharacterIterator(text);
        Node root = nextValue(iter);
        q.add(root);
        
        while (!q.isEmpty() && iter.current() != StringCharacterIterator.DONE) {
            Node next = q.poll();
            
            if (next != null) {
                Node left = nextValue(iter);
                Node right = nextValue(iter);
            
                next.left = left;
                next.right = right;
                
                q.add(left);
                q.add(right);
            }
        }
        
        return root;
    }
}

In [2]:
class NodeTest {
    public static void main() {
        Node node = Node.parseNode("5,4,7,3,#,2,#,-1,#,9");
        System.out.println(node);
    }
}
NodeTest.main();

5,4,7,3,#,2,#,-1,#,9,#,#,#,#,#,


# BFS (Level-Order Traversal)

1. Use a queue and add children of node as you process that node.
1. Null nodes need not be added to the tree.
   - It is not necessary to fill out with null values because the loop doesn't care about skipping
1. `O(n)` time and `O(n)` space

In [17]:
class BFSDemo {
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        Queue<Node> queue = new LinkedList<>();
        queue.add(root);
        while (!queue.isEmpty()) {
            Node next = queue.poll();
            System.out.println(next.value);
            if (next.left != null) queue.add(next.left);
            if (next.right != null) queue.add(next.right);
        }
    }
}
BFSDemo.main();

1
2
3
4
5
6
7


# DFS (Iterative, Pre-Order)

This code is eactly like __BFS__ above, but with a stack instead of a queue (and adding children in reverse order to compensate).

Technically , it is __pre-order DFS__ because we print the value of the parent before pushing the children.

Note that it goes all the way down the left side and then comes back up 1 level and back down, and so on, similar ot __backtracking__.

This algorithm is `O(n)` time and `O(depth)` space because the stack only gets filled out according to the current path and then backtracked up the tree.

In [19]:
class DFSDemo {
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        Stack<Node> stack = new Stack<>();
        stack.push(root);
        while (!stack.isEmpty()) {
            Node next = stack.pop();
            System.out.println(next.value);
            if (next.right != null) stack.push(next.right);
            if (next.left != null) stack.push(next.left);
        }
    }
}
DFSDemo.main();

1
2
4
5
3
6
7


# DFS (Recursive, Pre-Order)

This has the same order and complexities as the iterative version, but is more concise and intuitive.  There is likely a constant factor performance reduction from using the __program stack__ instead of a stack data structure.

In [20]:
class DFSDemo {
    private static void visit(Node node) {
        if (node == null) return;
        
        System.out.println(node.value);
        visit(node.left);
        visit(node.right);
    }
    
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        visit(root);
    }
}
DFSDemo.main();

1
2
4
5
3
6
7


# DFS (Recursive, Post-Order)

Just by moving where we print the current value relative to the recursions, we get a new order.

NOTE: even though the order of printing is different, the structure of the stack frames and the complexities is still the same (ignoring pruning that you might do).

The order of this one is a bubbling up from leftmost lowest children to their parents, going to the next leftmost lowest children in order to bubble everything we need at each node, until we've bubbled back up to root.

In [21]:
class DFSDemo {
    private static void visit(Node node) {
        if (node == null) return;
        
        visit(node.left);
        visit(node.right);
        System.out.println(node.value);
    }
    
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        visit(root);
    }
}
DFSDemo.main();

4
5
2
6
7
3
1


# DFS (Recursive, In-Order)

Similar to post-order above, we've changed the order but not the stack structure or complexity (ignoring pruning).

The order of this one is like a binary search tree.  All left tree children show up before a given node and then all right tree children.

In [22]:
class DFSDemo {
    private static void visit(Node node) {
        if (node == null) return;
        
        visit(node.left);
        System.out.println(node.value);
        visit(node.right);
    }
    
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        visit(root);
    }
}
DFSDemo.main();

4
2
5
1
6
3
7


# DFS (Iterative, Post-Order, Version 1)

One way to convert Post-Order DFS to iterative is to save a flag marking whether we're visiting the node to get its children vs. to get its value on a parallel stack structure.

This is sort of simulating what the recursive version does implicitly (because the instruction pointer saves where you came from in the calling function).

In this example, __two parallel stacks__ are used to do that, but an equivalent way to do it would be to make some sort of stack record class that wraps a node and the flag together.

In [2]:
class DFSDemo {
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        Stack<Node> stack = new Stack<>();
        Stack<Boolean> expandChildren = new Stack<>();
        stack.push(root);
        expandChildren.push(true);
        
        while (!stack.isEmpty()) {
            Node next = stack.pop();
            boolean expandChildrenNext = expandChildren.pop();
            
            if (expandChildrenNext) {
                stack.push(next);
                expandChildren.push(false);
                if (next.right != null) {
                    stack.push(next.right);
                    expandChildren.push(true);
                }
                if (next.left != null) {
                    stack.push(next.left);
                    expandChildren.push(true);
                }
            }
            else {
                System.out.println(next.value);
            }
        }
    }
}
DFSDemo.main();

4
5
2
6
7
3
1


# DFS (Iterative, Post-Order, Version 2)

This version uses __2 stacks__ as well, but they are __in series__ instead of parallel.

Although it looks like you're just reversing pre-order DFS, you are not.  A key difference in this version is that we push __left and then right__, instead of right and then left when we get the children.

A disadvantage of this approach is that you __lose the space benefits__ of DFS.  You end up with space `O(n)` like BFS instead of `O(depth)` like DFS.

In [3]:
class DFSDemo {
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        Stack<Node> stack1 = new Stack<>();
        Stack<Node> stack2 = new Stack<>();
        stack1.push(root);
        
        while (!stack1.isEmpty()) {
            Node next = stack1.pop();
            stack2.push(next);
            
            if (next.left != null) stack1.push(next.left);
            if (next.right != null) stack1.push(next.right);
        }
        
        while (!stack2.isEmpty()) {
            System.out.println(stack2.pop().value);
        }
    }
}
DFSDemo.main();

4
5
2
6
7
3
1


# DFS (Iterative, In-Order, Version 1)

This is __exactly like the Post-Order__ iterative verison 1 but you __rearrange the lines__ that add children to the stack so that the parent gets added the 2nd time in between the children.

In [4]:
class DFSDemo {
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        Stack<Node> stack = new Stack<>();
        Stack<Boolean> expandChildren = new Stack<>();
        stack.push(root);
        expandChildren.push(true);
        
        while (!stack.isEmpty()) {
            Node next = stack.pop();
            boolean expandChildrenNext = expandChildren.pop();
            
            if (expandChildrenNext) {
                if (next.right != null) {
                    stack.push(next.right);
                    expandChildren.push(true);
                }
                stack.push(next);
                expandChildren.push(false);
                if (next.left != null) {
                    stack.push(next.left);
                    expandChildren.push(true);
                }
            }
            else {
                System.out.println(next.value);
            }
        }
    }
}
DFSDemo.main();

4
2
5
1
6
3
7


# DFS (Iterative, In-Order, Version 2)

This has a particular pattern that looks strange but works out.

You started by pushing all the way down the left edge of the tree, then as you unwind each level back up the stack, you visit that node, then go to the right and push all the way down the left again.

It's a less direct way of visiting the left subtree, then current node, then right subtree like the recursive version.

In [9]:
class DFSDemo {
    public static void main() {
        Node root = Node.parseNode("1,2,3,4,5,6,7"); // 3 solid levels
        
        Stack<Node> stack = new Stack<>();
        Node current = root;

        while (current != null || !stack.isEmpty()) {
            // Reach the left most Node of the current Node
            while (current != null) {
                stack.push(current);
                current = current.left;
            }

            // Current must be null at this point
            current = stack.pop();
            System.out.println(current.value);

            // We have visited the node and its left subtree. Now, it's right subtree's turn
            current = current.right;
        }
    }
}
DFSDemo.main();

4
2
5
1
6
3
7


# Serialization

## BFS

See __Basic Setup__ above for the code for serializing and deserializing a tree using BFS order.

Compared to DFS, this way of serializing uses more space and time, but it is more __human readable__.

## DFS (Pre-Order)

Serialization works by visiting in pre-order DFS order as usual and adding # for null entries (without visiting their non-existent children).

Deserialization works by consuming the first number to make the root, then the next few to make the left subtree, and the rest to make the right subtree, recursively.

In [14]:
class DFSDemo {
    private static String serialize(Node root) {
        StringBuilder output = new StringBuilder();
        Stack<Node> stack = new Stack<>();
        stack.push(root);

        while (!stack.isEmpty()) {
            Node next = stack.pop();

            if (next == null) {
                output.append('#');
            }
            else {
                output.append(next.value);
                stack.push(next.right);
                stack.push(next.left);
            }

            output.append(',');
        }

        return output.toString();
    }
    
    private static Node deserialize(StringCharacterIterator iter) {
        if (iter.current() == '#') {
            iter.next();
            iter.next();
            return null;
        }

        Node newNode = new Node(0);
        int sign = iter.current() == '-' ? -1 : 1;
        if (sign == -1) {
            iter.next();
        }
        while (iter.current() != ',') {
            newNode.value = 10 * newNode.value + iter.current() - '0';
            iter.next();
        }
        iter.next();
        newNode.value *= sign;

        newNode.left = deserialize(iter);
        newNode.right = deserialize(iter);

        return newNode;
    }

    // Decodes your encoded data to tree.
    private static Node deserialize(String data) {
        return deserialize(new StringCharacterIterator(data));
    }
    
    public static void main() {
        Node node = Node.parseNode("5,4,7,3,#,2,#,-1,#,9");
        
        System.out.println(serialize(node));
        
        node = deserialize(serialize(node));
        
        System.out.println(node);
    }
}
DFSDemo.main();

5,4,3,-1,#,#,#,#,7,2,9,#,#,#,#,
5,4,7,3,#,2,#,-1,#,9,#,#,#,#,#,


# String Backtracking (DFS)

This is not strictly considered either pre or post order because it's not a true tree.  The reason it's thought of as DFS is because you go all the way down the whole string before returning any result, using stack recursion.

WARNING: if you don't reset `text[i]` to `?` when you're done, then when you come back on a future recursion you will be stuck at 0 here instead of the wildcard.  This is the __backtracking__ step.

The __time complexity__ is `O(2^w)` where w is the # of wildcards (potentially up to n) - because of the branching factor of 2.

The __space complexity__ is `O(w)` due to the stack recursion down all the wildcards, but since you make a copy of the string, it's more accurate to say `O(n)`.

In [24]:
class BacktrackDemo {
    private static void fillWildcards(char[] text, int index, List<String> results) {
        boolean found = false;
        for (int i = index; i < text.length; i++) {
            char c = text[i];
            if (c == '?') {
                text[i] = '0';
                fillWildcards(text, index+1, results);
                text[i] = '1';
                fillWildcards(text, index+1, results);
                found = true;
                text[i] = '?';  // backtrack
                
                break;
            }
        }
        if (!found) {
            results.add(new String(text));
        }
    }
    
    public static List<String> fillWildcards(String text) {
        List<String> ret = new ArrayList<>();
        fillWildcards(text.toCharArray(), 0, ret);
        
        return ret;
    }
    
    public static void main() {
        System.out.println(fillWildcards("10?10?"));
    }
}
BacktrackDemo.main();

[100100, 100101, 101100, 101101]


# String Implicit Backtracking (DFS)

Above, we explicitly reset values back to `?` when we're going back up the stack.  If you instead have an algorithm that passes an array length as part of the recursion (such as an incrementally increasing prefix size), then you can backtrack just by ignoring the clobbered part after that index and clobbering it again as you go back down the stack.

# StringCharacterIterator

`java.text.StringCharacterIterator` is a way to consume a string char by char in a recursive helper function without having to pass back that information to the caller.